In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport utils
%aimport Networks

In [13]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from os import path
from sklearn.preprocessing import StandardScaler
from utils import load_embeddings_and_ids, User, concatenate_featmats
from Networks import VBPR_Network_Evaluation

In [4]:
# use a single GPU because we want to be nice with other people :)
os.environ["CUDA_VISIBLE_DEVICES"]="1"

###  Load pre-trained image embeddings

In [8]:
resnet50 = load_embeddings_and_ids('/mnt/workspace/Ugallery/ResNet50/', 'flatten_1.npy', 'ids')
# alexnet = load_embeddings_and_ids('/mnt/workspace/Ugallery/AlexNet/', 'fc7.npy', 'ids.npy')
# inceptionv3 = load_embeddings_and_ids('/mnt/workspace/Ugallery/InceptionV3/', 'avg_pool.npy', 'ids')

###  Concatenate embeddings + z-score normalization

In [9]:
embedding_list = [
    resnet50,
#     alexnet,
#     inceptionv3,
]

In [11]:
artwork_ids_set = set()
for embedding in embedding_list:
    artwork_ids_set.update(embedding['index2id'])
artwork_ids = list(artwork_ids_set)
artwork_id2index = {_id:i for i,_id in enumerate(artwork_ids)}
n_artworks = len(artwork_ids)
n_artworks

13297

In [14]:
featmat_list = [tmp['featmat'] for tmp in embedding_list]
id2index_list = [tmp['id2index'] for tmp in embedding_list]
concat_featmat = concatenate_featmats(artwork_ids, featmat_list, id2index_list)

In [15]:
concat_featmat = StandardScaler().fit_transform(concat_featmat)

In [21]:
concat_featmat[0]

array([-0.50733465, -0.77183085, -0.38606572, ..., -0.17101007,
       -0.70491479, -0.59356332])

In [22]:
concat_featmat.shape

(13297, 2048)

### Load user list from transactions

In [23]:
sales_df = pd.read_csv('./data/valid_sales.csv')
user_ids = sales_df.customer_id.unique()
n_users = len(user_ids)

In [24]:
n_users

2919

### Load saved VBPR network and compute tensors

In [32]:
all_indexes = list(range(n_artworks))

In [47]:
MODEL_PATH = '/mnt/workspace/pamessina_models/ugallery/VBPR/v12(10M-400K,rsnt50,u(200)i(100l+100v),+p-np,fg.6,vcf.1)/'
MODEL_PATH

'/mnt/workspace/pamessina_models/ugallery/VBPR/v12(10M-400K,rsnt50,u(200)i(100l+100v),+p-np,fg.6,vcf.1)/'

In [48]:
with tf.Graph().as_default():
    network = VBPR_Network_Evaluation(
        n_users=n_users,
        n_items=n_artworks,
        user_latent_dim=200,
        item_latent_dim=100,
        item_visual_dim=100,
        pretrained_dim=2048,
    )
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, tf.train.latest_checkpoint(MODEL_PATH))
        user_latent_vectors = network.get_user_latent_vectors(sess)
        item_vectors, item_biases = network.get_item_final_vector_bias(sess, concat_featmat, all_indexes)

INFO:tensorflow:Restoring parameters from /mnt/workspace/pamessina_models/ugallery/VBPR/v12(10M-400K,rsnt50,u(200)i(100l+100v),+p-np,fg.6,vcf.1)/


In [49]:
item_vectors.shape, item_biases.shape

((13297, 200), (13297,))

In [50]:
user_latent_vectors.shape

(2919, 200)

In [51]:
dir_path = '/mnt/workspace/Ugallery/VBPR/v12(10M-400K,rsnt50,u(200)i(100l+100v),+p-np,fg.6,vcf.1)/'
os.makedirs(dir_path, exist_ok=True)

# users
user_latent_vectors.dump(dir_path + "user_vectors.npy")
with open(dir_path + 'user_ids', 'w') as f:
    for _id in user_ids:
        f.write('%d\n' % _id)
        
# items
item_vectors.dump(dir_path + "item_vectors.npy")
item_biases.dump(dir_path + "item_biases.npy")
with open(dir_path + 'items_ids', 'w') as f:
    for _id in artwork_ids:
        f.write('%d\n' % _id)